In [ ]:
# 这个repo模型被我修改过

In [1]:
import os
import sys

_r = os.getcwd().split('/')
_p = '/'.join(_r[:_r.index('PFLD-pytorch')+1])
print('Change dir from %s to %s' % (os.getcwd(), _p))
os.chdir(_p)
sys.path.append(_p)

Change dir from /workspace/PFLD-pytorch to /workspace/PFLD-pytorch


In [2]:
import argparse
import time

import numpy as np
import torch
import torchvision
from torchvision import transforms
import cv2

from models.pfld import PFLDInference, AuxiliaryNet
from mtcnn.detector import detect_faces, show_bboxes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from matplotlib import pyplot as plt
import cv2

img = cv2.imread('./pyncnn/test.jpeg')
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# plt.imshow(img)
# plt.title('my picture')
# plt.show()

In [4]:
model_path = './checkpoint/snapshot/checkpoint.pth.tar'
checkpoint = torch.load(model_path, map_location=device)
plfd_backbone = PFLDInference().to(device)
plfd_backbone.load_state_dict(checkpoint['plfd_backbone'])
plfd_backbone.eval()
plfd_backbone = plfd_backbone.to(device)
transform = transforms.Compose([transforms.ToTensor()])

height, width = img.shape[:2]
bounding_boxes, landmarks = detect_faces(img)
for box in bounding_boxes:
    score = box[4]
    x1, y1, x2, y2 = (box[:4]+0.5).astype(np.int32)
    w = x2 - x1 + 1
    h = y2 - y1 + 1

    size = int(max([w, h])*1.1)
    cx = x1 + w//2
    cy = y1 + h//2
    x1 = cx - size//2
    x2 = x1 + size
    y1 = cy - size//2
    y2 = y1 + size

    dx = max(0, -x1)
    dy = max(0, -y1)
    x1 = max(0, x1)
    y1 = max(0, y1)

    edx = max(0, x2 - width)
    edy = max(0, y2 - height)
    x2 = min(width, x2)
    y2 = min(height, y2)
    cropped = img[y1:y2, x1:x2]
    if (dx > 0 or dy > 0 or edx > 0 or edy > 0):
        cropped = cv2.copyMakeBorder(cropped, dy, edy, dx, edx, cv2.BORDER_CONSTANT, 0)

    cropped = cv2.resize(cropped, (112, 112))

    input = cv2.resize(cropped, (112, 112))
    input = cv2.cvtColor(input, cv2.COLOR_BGR2RGB)
    input = transform(input).unsqueeze(0).to(device)
    _, landmarks = plfd_backbone(input)
    pre_landmark = landmarks[0]
    pre_landmark = pre_landmark.cpu().detach().numpy().reshape(-1, 2) * [size, size]
    for (x, y) in pre_landmark.astype(np.int32):
        cv2.circle(img, (x1 + x, y1 + y), 1, (0, 0, 255))

RuntimeError: Error(s) in loading state_dict for PFLDInference:
	Missing key(s) in state_dict: "conv2_dw.0.weight", "conv2_dw.1.weight", "conv2_dw.1.bias", "conv2_dw.1.running_mean", "conv2_dw.1.running_var", "conv2_dw.3.weight", "conv2_dw.4.weight", "conv2_dw.4.bias", "conv2_dw.4.running_mean", "conv2_dw.4.running_var". 
	Unexpected key(s) in state_dict: "conv2.weight", "bn2.weight", "bn2.bias", "bn2.running_mean", "bn2.running_var", "bn2.num_batches_tracked". 

In [ ]:
plt.imshow(img)
plt.title('my picture')
plt.show()